In [2]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.cluster import KMeans
import logging
import math
import numpy as np
import torch
from torch.nn import ModuleList
from functools import partial
from typing import List, Tuple, Dict, Optional, Any
from torchvision.transforms import Compose, RandomApply
from torchvision.transforms import functional as F
from torchvision.transforms.transforms import _setup_angle, _check_sequence_input
from torch import Tensor
from collections import defaultdict, deque
from pathlib import Path
from torch import nn
from PIL import ImageFilter, ImageOps, Image, ImageDraw
import glob
import os
def read_assets_from_h5( h5_path: str) -> tuple:
    '''Read the assets frdom the h5 file'''
    assets = {}
    attrs = {}
    with h5py.File(h5_path, 'r') as f:
        for key in f.keys():
            assets[key] = f[key][:]
            if f[key].attrs is not None:
                attrs[key] = dict(f[key].attrs)
    return assets, attrs

In [ ]:

h5_file_list = feature_path= [f for f in Path('/data4/embedding').rglob('*.h5')]
#print(h5_file_list)
for file in h5_file_list:
    try:
        data=read_assets_from_h5(file)[0]
        feature = data['features']
        coords = data['coords']

        if len(coords.shape) != 2:
            print('remove the file:', file)
        if feature.shape[0] <= 8:
            print('remove the file:', file)
            os.remove(file)
    except:
        print('can not open:', file)
        #os remove
        os.remove(file)
 
    

remove the file: /data4/embedding/private_chunk_3/FMBC/S12867.h5
